In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
from datetime import datetime
import time

In [3]:
class TradingStrategy:
    def __init__(self,name):
        self.__name = name            # Name Variable is made private 
    def generatesignal(self,price_data):
         print("This class has to be overridden")
         return "HOLD"
    @property
    def name(self):
        return self.__name            # we can directly acces name as variable not as function using this 

In [4]:
class SmaTradingStrategy(TradingStrategy):
    def __init__(self,swindow,lwindow):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__("Sma Crossover Strategy")
    def generatesignal(self,price_data):
        if len(price_data) <  self.__lwindow :
            return "HOLD"
        short_avg = sum(price_data[-self.__swindow:])/self.__swindow         # Calculating simple moving avrage
        long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow
        if short_avg < long_avg :
            return "Sell"
        elif short_avg > long_avg :
            return "Buy"
        else:
            return "Hold"
    @property
    def swindow(self):
        return self.__swindow 

    @property
    def lwindow(self):
        return self.__lwindow          

In [5]:
class Trade:
    def __init__(self,strat_name,price,amount,signal):
        self.__strat_name = strat_name
        self.__price = price
        self.__amount = amount
        self.__signal = signal
        self.__timestamp = datetime.now()
    def execute(self):
        if self.__signal == "Hold":
            print(f"No Trade now to be executed as signal is Hold at time {self.__timestamp}")
        else:
            print(f"Executed a {self.__signal} trade using strategy {self.__strat_name} at price {self.__price} with amount {self.__amount} at time {self.__timestamp}")
    @property
    def strat_name(self):
        return self.__strat_name
    @property
    def price(self):
        return self.__price 
    @property
    def amount(self):
        return self.__amount 
    @property
    def signal(self):
        return self.__signal 

In [6]:
class MockApiAccount:
    def __init__(self,balance):
        self.__balance = balance
    def execute(self,trade,price):
        if trade.signal=="Buy":
            if trade.amount<=self.__balance:
                self.__balance-= trade.amount
                print(f"Executed a {trade.signal} with amount {trade.amount} with remaning balance of {self.__balance}")
            else:
                print(f"Not Enough Balance: you need {trade.amount-self.__balance} more to execute this trade")
        elif trade.signal=="Sell":
            self.__balance+= trade.amount
            print(f"Executed a {trade.signal} with amount {trade.amount} with remaning balance of {self.__balance}")

    @property
    def balance(self):
        return self.__balance
            
            
            

In [22]:
class TradingData:
    def __init__(self,api,strategy,symbol):
        self.__api= api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []
    def fetch_price_data(self):
        data = yf.download( tickers = self.__symbol, period = "1D",interval = "1m")
        if not data.empty:
            price = data['Close'].iloc[-1]           # Fetching the last closing price
            self.__price_data.append(price)
            if(len(self.__price_data)> self.__strategy.lwindow):
                print(f"Fetched price data: {self.__price_data}")
            else:
                print("No data fetched")
    def run(self):
        self.fetch_price_data()
        signal = self.__strategy.generatesignal(self.__price_data)
        print(f"Generated Sinal: {signal}")
        if signal in ["Sell","Buy"]:
            trade = Trade(self.__strategy.name,signal,1)
            trade.execute()
            self.__api.execute(trade,self.__price_data[-1])
    @property
    def api(self):
        return self.__api

    @property
    def strategy(self):
        return self.__strategy

    @property
    def symbol(self):
        return self.__symbol

    @property
    def balance(self):
        return self.__price_data

    

In [24]:
symbol = 'AAPL'
api = MockApiAccount(100000)
strategy = SmaTradingStrategy(1,5)
tradedata = TradingData(api,strategy,symbol)
for i in range(3):
    tradedata.run()
    print(f"Remaining Balance: {api.balance}")
    time.sleep(2)

C:\Users\91889\AppData\Local\Temp\ipykernel_17648\3686980608.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download( tickers = self.__symbol, period = "1D",interval = "1m")
[*********************100%***********************]  1 of 1 completed


No data fetched
Generated Sinal: HOLD
Remaining Balance: 100000


C:\Users\91889\AppData\Local\Temp\ipykernel_17648\3686980608.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download( tickers = self.__symbol, period = "1D",interval = "1m")
[*********************100%***********************]  1 of 1 completed


No data fetched
Generated Sinal: HOLD
Remaining Balance: 100000


C:\Users\91889\AppData\Local\Temp\ipykernel_17648\3686980608.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download( tickers = self.__symbol, period = "1D",interval = "1m")
[*********************100%***********************]  1 of 1 completed


No data fetched
Generated Sinal: HOLD
Remaining Balance: 100000


In [ ]:
check = SmaTradingStrategy(3,5)
signal = check.generatesignal([1,2,3,4,5,10,20])
strat_name = check.name
trade = Trade(strat_name,100,100000,signal)
mocktest = MockApiAccount(100)
mocktest.execute(trade,500)
trading = TradingData(mocktest,check,"AAPL")
print(trading.fetch_price_data().head(5))